# 🔬 ISIC Cilt Lezyonu Analizi - ÇALIŞAN VERSİYON

**Mustafa Engin Dalgıç | 254309502 | Üsküdar Üniversitesi**

---

## ▶️ ÇALIŞTIRMA:

1. **Connect** butonuna tıklayın (sağ üst) ⚡
2. **Runtime > Run all**
3. İşlem başlayacak! ✅

---

In [ ]:
print("="*70)
print("🚀 ISIC CİLT LEZYONU ANALİZİ")
print("="*70)
print("\nProgram başlıyor...\n")

In [ ]:
# ==================== KÜTÜPHANELER ====================
import sys, os, warnings
warnings.filterwarnings('ignore')

print("📦 Kütüphaneler yükleniyor...\n")

# OpenCV
try:
    import cv2
except:
    print("⚠️ OpenCV kuruluyor...")
    !pip install opencv-python -q
    import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request, zipfile

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

print(f"✅ OpenCV {cv2.__version__}")
print(f"✅ NumPy {np.__version__}")
print(f"✅ Pandas {pd.__version__}")
print("\n✅ Kütüphaneler hazır!\n")

In [ ]:
# ==================== PROJE İNDİRME ====================
print("="*70)
print("📥 GITHUB'DAN PROJE İNDİRİLİYOR")
print("="*70 + "\n")

# Temizlik
!rm -rf UU_DIP_ISIC_SKIN_LESION_ANALYSIS 2>/dev/null
!rm -f project.zip 2>/dev/null

# ZIP indir
zip_url = "https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS/archive/refs/heads/main.zip"

print("📦 ZIP indiriliyor...")
urllib.request.urlretrieve(zip_url, "project.zip")

print("📂 ZIP açılıyor...")
with zipfile.ZipFile("project.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

# Klasör adını düzelt
if os.path.exists("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main"):
    !mv UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main UU_DIP_ISIC_SKIN_LESION_ANALYSIS

print("✅ İndirme tamamlandı!")

# Klasöre geç
%cd UU_DIP_ISIC_SKIN_LESION_ANALYSIS

print("\n📁 Klasör içeriği:")
!ls -lh | head -20

In [ ]:
# ==================== VERİ SETİ DETAYLI KONTROL ====================
print("\n" + "="*70)
print("🔍 VERİ SETİ KONTROLÜ")
print("="*70 + "\n")

DATA_PATH = "ISIC"

print(f"📂 ISIC klasörü kontrolü:")
print(f"   Var mı? {os.path.exists(DATA_PATH)}")
print(f"   Dizin mi? {os.path.isdir(DATA_PATH)}")

if os.path.exists(DATA_PATH):
    print(f"\n📋 ISIC içeriği:")
    items = os.listdir(DATA_PATH)
    print(f"   Toplam öğe: {len(items)}")
    
    # İlk 20 öğeyi göster
    print(f"\n   İlk 20 öğe:")
    for i, item in enumerate(items[:20], 1):
        item_path = os.path.join(DATA_PATH, item)
        if os.path.isdir(item_path):
            print(f"   {i:2d}. 📁 {item}/")
        else:
            size = os.path.getsize(item_path) / 1024
            print(f"   {i:2d}. 📄 {item} ({size:.1f} KB)")
    
    if len(items) > 20:
        print(f"   ... ve {len(items)-20} tane daha")
else:
    print("   ❌ ISIC klasörü bulunamadı!")

# Görüntüleri tara
print(f"\n🔍 Görüntü dosyaları taranıyor...")
image_files = []

if os.path.exists(DATA_PATH):
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            # Tüm uzantıları dene (.jpg, .JPG, .jpeg, .JPEG, .png, .PNG)
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                full_path = os.path.join(root, file)
                image_files.append(full_path)

print(f"\n📊 SONUÇ:")
if len(image_files) == 0:
    print("   ❌ HİÇBİR GÖRÜNTÜ BULUNAMADI!")
    print("\n   💡 Bu GitHub reposunda ISIC klasöründe görüntü olmalı.")
    print("   Lütfen GitHub'ı kontrol edin.")
else:
    print(f"   ✅ {len(image_files)} GÖRÜNTÜ BULUNDU!")
    print(f"\n   İlk 5 dosya:")
    for i, f in enumerate(image_files[:5], 1):
        print(f"   {i}. {f}")

In [ ]:
# ==================== VERİ YÜKLEME ====================
if len(image_files) == 0:
    print("\n❌ Görüntü olmadığı için devam edilemiyor!")
    print("Lütfen GitHub reposuna ISIC klasörüne görüntü ekleyin.")
else:
    print("\n" + "="*70)
    print("📊 VERİ YÜKLEME")
    print("="*70 + "\n")
    
    # İlk 30 görüntüyü yükle (hızlı test)
    sample_size = min(30, len(image_files))
    print(f"📦 İlk {sample_size} görüntü yükleniyor...\n")
    
    data = []
    for i, img_path in enumerate(image_files[:sample_size], 1):
        try:
            img = Image.open(img_path)
            w, h = img.size
            
            if img.mode == 'RGB':
                ch = 3
            elif img.mode == 'L':
                ch = 1
            else:
                ch = len(img.getbands())
            
            sz = os.path.getsize(img_path) / 1024
            
            data.append({
                'filename': os.path.basename(img_path),
                'filepath': img_path,
                'width': w,
                'height': h,
                'channels': ch,
                'file_size_kb': round(sz, 2)
            })
            
            if i % 10 == 0:
                print(f"   {i}/{sample_size} yüklendi...")
                
        except Exception as e:
            print(f"   ⚠️ Atlandı: {os.path.basename(img_path)}")
    
    df = pd.DataFrame(data)
    print(f"\n✅ {len(df)} görüntü başarıyla yüklendi!")
    print("\n📋 İlk 5 satır:")
    display(df.head())

In [ ]:
# ==================== İSTATİSTİKLER ====================
if len(image_files) > 0 and len(df) > 0:
    print("\n" + "="*70)
    print("📊 İSTATİSTİKLER")
    print("="*70 + "\n")
    
    print(f"📐 Çözünürlük:")
    print(f"   Ortalama: {df['width'].mean():.0f}x{df['height'].mean():.0f} px")
    print(f"   Min: {df['width'].min()}x{df['height'].min()}")
    print(f"   Max: {df['width'].max()}x{df['height'].max()}")
    
    print(f"\n💾 Dosya Boyutu:")
    print(f"   Ortalama: {df['file_size_kb'].mean():.2f} KB")
    print(f"   Toplam: {df['file_size_kb'].sum()/1024:.2f} MB")

In [ ]:
# ==================== GÖRÜNTÜ GÖSTERİMİ ====================
if len(image_files) > 0 and len(df) > 0:
    print("\n" + "="*70)
    print("🖼️ GÖRÜNTÜ GÖSTERİMİ")
    print("="*70 + "\n")
    
    def resize_img(img, max_size=400):
        h, w = img.shape[:2]
        if max(h, w) > max_size:
            scale = max_size / max(h, w)
            return cv2.resize(img, None, fx=scale, fy=scale)
        return img
    
    # İlk 3 görüntü
    n = min(3, len(df))
    
    fig, axes = plt.subplots(n, 2, figsize=(10, n*2.5))
    if n == 1:
        axes = axes.reshape(1, -1)
    
    fig.suptitle('✅ RGB vs Grayscale Karşılaştırma', fontsize=14, fontweight='bold')
    
    for idx in range(n):
        row = df.iloc[idx]
        img = cv2.imread(row['filepath'])
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb = resize_img(img_rgb)
        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        
        axes[idx, 0].imshow(img_rgb)
        axes[idx, 0].set_title(f"RGB - {row['filename'][:25]}")
        axes[idx, 0].axis('off')
        
        axes[idx, 1].imshow(img_gray, cmap='gray')
        axes[idx, 1].set_title(f"Grayscale - {row['width']}x{row['height']}")
        axes[idx, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Görüntüler gösterildi!")

In [ ]:
# ==================== HİSTOGRAM ====================
if len(image_files) > 0 and len(df) > 0:
    print("\n" + "="*70)
    print("📊 HİSTOGRAM ANALİZİ")
    print("="*70 + "\n")
    
    # İlk görüntü
    first = df.iloc[0]
    img = cv2.imread(first['filepath'])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = resize_img(img_rgb, 400)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    # Histogram
    hist_r = cv2.calcHist([img_rgb], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([img_rgb], [1], None, [256], [0, 256])
    hist_b = cv2.calcHist([img_rgb], [2], None, [256], [0, 256])
    hist_gray = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    fig.suptitle('Histogram Analizi', fontsize=14, fontweight='bold')
    
    axes[0].plot(hist_r, 'r-', alpha=0.7, label='Red')
    axes[0].plot(hist_g, 'g-', alpha=0.7, label='Green')
    axes[0].plot(hist_b, 'b-', alpha=0.7, label='Blue')
    axes[0].set_title('RGB Histogram')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    axes[1].plot(hist_gray, 'k-')
    axes[1].set_title('Grayscale Histogram')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Histogram tamamlandı!")

In [ ]:
# ==================== CLAHE ====================
if len(image_files) > 0 and len(df) > 0:
    print("\n" + "="*70)
    print("✨ CLAHE - KONTRAST İYİLEŞTİRME")
    print("="*70 + "\n")
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_clahe = clahe.apply(img_gray)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle('CLAHE - Kontrast İyileştirme', fontsize=14, fontweight='bold')
    
    axes[0].imshow(img_gray, cmap='gray')
    axes[0].set_title('Orijinal')
    axes[0].axis('off')
    
    axes[1].imshow(img_clahe, cmap='gray')
    axes[1].set_title('CLAHE Uygulanmış')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ CLAHE tamamlandı!")

---

## 🎉 ANALİZ TAMAMLANDI!

### ✅ Başarıyla Tamamlanan:
1. Veri seti yükleme
2. İstatistikler
3. RGB vs Grayscale
4. Histogram analizi
5. CLAHE uygulaması

### 🚀 TAM PROGRAM:

45+ grafik için:

In [ ]:
if len(image_files) > 0:
    print("\n" + "="*70)
    print("🚀 TAM PROGRAM BAŞLIYOR")
    print("="*70)
    print("\n⏱️ Tahmini süre: 10-15 dakika")
    print("📊 45+ grafik oluşturulacak\n")
    
    !python isic_project.py

---

### 💬 İletişim

**Mustafa Engin Dalgıç**  
📧 engindalgic86@gmail.com  
🎓 Üsküdar Üniversitesi  
🔗 [GitHub](https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS)

⭐ **Projeyi beğendiyseniz yıldız vermeyi unutmayın!** ⭐